## leinforcement pyform


In [1]:
#!/usr/bin/python
import numpy as np
from collections import deque
from tqdm import tqdm

from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
from gym import spaces
from gym.spaces.box import Box

import os
import PyFoam
import PyFoam.FoamInformation
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector
from PyFoam.Execution.BasicRunner import BasicRunner
from PyFoam.Basics.TemplateFile import TemplateFile
import shlex,sys,json
import re
from pathlib import Path

In [2]:
#PyFOAM基本コマンド 

In [2]:
# set directory
CASE = SolutionDirectory("../aircond2")

In [3]:
os.system("./Allclean")

0

In [4]:
os.system('./Makemesh')

0

In [7]:
# OpenFOAMのコマンドを実行
args=shlex.split("buoyantBoussinesqPimpleFoam -case " + CASE.name)
buoyant=BasicRunner(args,silent=True)
summary=buoyant.start()
buoyant.runOK()

True

# 以下、棒倒しのDQN

In [6]:
# パッケージのimport
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym


In [7]:
# 動画の描画関数の宣言
# 参考URL http://nbviewer.jupyter.org/github/patrickmineault
# /xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    anim.save('movie_cartpole_DQN.mp4')  # 動画のファイル名と保存です
    display(display_animation(anim, default_mode='loop'))
    

In [8]:
# 本コードでは、namedtupleを使用します。
# namedtupleを使うことで、値をフィールド名とペアで格納できます。
# すると値に対して、フィールド名でアクセスできて便利です。
# https://docs.python.jp/3/library/collections.html#collections.namedtuple
# 以下は使用例です

from collections import namedtuple

Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)

print(Tr_object)  # 出力：tr(name_a='名前Aです', value_b=100)
print(Tr_object.value_b)  # 出力：100


tr(name_a='名前Aです', value_b=100)
100


In [9]:
# namedtupleを生成
from collections import namedtuple

Transition = namedtuple(
    'Transition', ('state', 'action', 'next_state', 'reward'))


In [37]:
# 定数の設定
ENV = 'CartPole-v0'  # 使用する課題名
GAMMA = 0.99  # 時間割引率
MAX_STEPS = 200  # 1試行のstep数
NUM_EPISODES = 500  # 最大試行回数


In [11]:
# 経験を保存するメモリクラスを定義します


class ReplayMemory:

    def __init__(self, CAPACITY):
        self.capacity = CAPACITY  # メモリの最大長さ
        self.memory = []  # 経験を保存する変数
        self.index = 0  # 保存するindexを示す変数

    def push(self, state, action, state_next, reward):
        '''transition = (state, action, state_next, reward)をメモリに保存する'''

        if len(self.memory) < self.capacity:
            self.memory.append(None)  # メモリが満タンでないときは足す

        # namedtupleのTransitionを使用し、値とフィールド名をペアにして保存します
        self.memory[self.index] = Transition(state, action, state_next, reward)

        self.index = (self.index + 1) % self.capacity  # 保存するindexを1つずらす

    def sample(self, batch_size):
        '''batch_size分だけ、ランダムに保存内容を取り出す'''
        return random.sample(self.memory, batch_size)

    def __len__(self):
        '''関数lenに対して、現在の変数memoryの長さを返す'''
        return len(self.memory)


In [12]:
# エージェントが持つ脳となるクラスです、DQNを実行します
# Q関数をディープラーニングのネットワークをクラスとして定義

import random
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

BATCH_SIZE = 32
CAPACITY = 10000


class Brain:
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions  # CartPoleの行動（右に左に押す）の2を取得

        # 経験を記憶するメモリオブジェクトを生成
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワークを構築
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)  # ネットワークの形を出力

        # 最適化手法の設定
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    def replay(self):
        '''Experience Replayでネットワークの結合パラメータを学習'''

        # -----------------------------------------
        # 1. メモリサイズの確認
        # -----------------------------------------
        # 1.1 メモリサイズがミニバッチより小さい間は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # -----------------------------------------
        # 2. ミニバッチの作成
        # -----------------------------------------
        # 2.1 メモリからミニバッチ分のデータを取り出す
        transitions = self.memory.sample(BATCH_SIZE)

        # 2.2 各変数をミニバッチに対応する形に変形
        # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
        # つまり、(state, action, state_next, reward)×BATCH_SIZE
        # これをミニバッチにしたい。つまり
        # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
        batch = Transition(*zip(*transitions))

        # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
        # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
        # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
        # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
        # catはConcatenates（結合）のことです。
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])

        # -----------------------------------------
        # 3. 教師信号となるQ(s_t, a_t)値を求める
        # -----------------------------------------
        # 3.1 ネットワークを推論モードに切り替える
        self.model.eval()

        # 3.2 ネットワークが出力したQ(s_t, a_t)を求める
        # self.model(state_batch)は、右左の両方のQ値を出力しており
        # [torch.FloatTensor of size BATCH_SIZEx2]になっている。
        # ここから実行したアクションa_tに対応するQ値を求めるため、action_batchで行った行動a_tが右か左かのindexを求め
        # それに対応するQ値をgatherでひっぱり出す。
        state_action_values = self.model(state_batch).gather(1, action_batch)

        # 3.3 max{Q(s_t+1, a)}値を求める。ただし次の状態があるかに注意。

        # cartpoleがdoneになっておらず、next_stateがあるかをチェックするインデックスマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None,
                                                    batch.next_state)))
        # まずは全部0にしておく
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        # 出力にアクセスし、max(1)で列方向の最大値の[値、index]を求めます
        # そしてそのQ値（index=0）を出力します
        # detachでその値を取り出します
        next_state_values[non_final_mask] = self.model(
            non_final_next_states).max(1)[0].detach()

        # 3.4 教師となるQ(s_t, a_t)値を、Q学習の式から求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # -----------------------------------------
        # 4. 結合パラメータの更新
        # -----------------------------------------
        # 4.1 ネットワークを訓練モードに切り替える
        self.model.train()

        # 4.2 損失関数を計算する（smooth_l1_lossはHuberloss）
        # expected_state_action_valuesは
        # sizeが[minbatch]になっているので、unsqueezeで[minibatch x 1]へ
        loss = F.smooth_l1_loss(state_action_values,
                                expected_state_action_values.unsqueeze(1))

        # 4.3 結合パラメータを更新する
        self.optimizer.zero_grad()  # 勾配をリセット
        loss.backward()  # バックプロパゲーションを計算
        self.optimizer.step()  # 結合パラメータを更新

    def decide_action(self, state, episode):
        '''現在の状態に応じて、行動を決定する'''
        # ε-greedy法で徐々に最適行動のみを採用する
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            self.model.eval()  # ネットワークを推論モードに切り替える
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)
            # ネットワークの出力の最大値のindexを取り出します = max(1)[1]
            # .view(1,1)は[torch.LongTensor of size 1]　を size 1x1 に変換します

        else:
            # 0,1の行動をランダムに返す
            action = torch.LongTensor(
                [[random.randrange(self.num_actions)]])  # 0,1の行動をランダムに返す
            # actionは[torch.LongTensor of size 1x1]の形になります

        return action


In [13]:
# CartPoleで動くエージェントクラスです、棒付き台車そのものになります


class Agent:
    def __init__(self, num_states, num_actions):
        '''課題の状態と行動の数を設定する'''
        self.brain = Brain(num_states, num_actions)  # エージェントが行動を決定するための頭脳を生成

    def update_q_function(self):
        '''Q関数を更新する'''
        self.brain.replay()

    def get_action(self, state, episode):
        '''行動を決定する'''
        action = self.brain.decide_action(state, episode)
        return action

    def memorize(self, state, action, state_next, reward):
        '''memoryオブジェクトに、state, action, state_next, rewardの内容を保存する'''
        self.brain.memory.push(state, action, state_next, reward)


In [5]:
# set action variable
# set 9 parameter

# speed is 0.1 0.3 0.5
# direction is -/8pi, -2/8pi, -3/8pi
# temperture is 18, 22, 26
SPEED = np.array([0.1,0.3,0.5])
DIRECTION = np.array([-1*np.pi/8, -2*np.pi/8,-3*np.pi/8])
TEMPERTURE = np.array([18,22,26])
#Ux = SPEED*np.cos(DIRECTION)
#Uy = SPEED*np.sin(DIRECTION)
#Ux**2+Uy**2  # check calculation
#U = np.array([Ux,Uy,np.zeros(3)])
#U = U.transpose()

In [6]:
Action_space= np.empty((0,3),float)
for i in range(len(SPEED)):
    for j in range(len(DIRECTION)):
        for k in range(len(TEMPERTURE)):
            Ux = SPEED[i]*np.cos(DIRECTION[j])
            Uy = SPEED[i]*np.sin(DIRECTION[j])
            Act = np.array([[Ux,Uy,TEMPERTURE[k]]])
            Action_space = np.append(Action_space,Act,axis=0)
            
Action_space.shape

(27, 3)

In [32]:
U_initial = np.array(ParsedParameterFile(CASE.initialDir() + '/U').content['internalField'])
U_latest = np.array(ParsedParameterFile(CASE.latestDir() + '/U').content['internalField'])
T_initial = np.array(ParsedParameterFile(CASE.initialDir() + '/T').content['internalField'])
T_latest = np.array(ParsedParameterFile(CASE.latestDir() + '/T').content['internalField'])

In [35]:
U_latest

array('uniform (0 0 0)', dtype=object)

In [8]:
U_initial.ndim  # 条件判定に使う

0

In [9]:
U_value = str(U_initial.all())
U_value

'uniform (0 0 0)'

In [10]:
U = U_value[8:].strip('()').split(' ')

In [11]:
a = np.array(list(map(int,U)))

In [12]:
np.tile(a,(nCells,1))

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [430]:
T_value = str(T_initial.all())

In [434]:
T = T_value[8:].strip('()').split(' ')

In [457]:
b = np.array(list(map(int,T)))

In [446]:
np.array(list(map(int,T)))

array([300])

In [459]:
np.tile(b,(nCells,1)).shape

(404, 1)

In [276]:
U_latest_xy = np.delete(U_latest, axis=1, obj=2)
T_latest_x = np.reshape(T_latest, [-1,1], order='F')

In [284]:
Observation = np.concatenate([U_latest_xy, T_latest_x],axis=1)

In [285]:
Observation

array([[-6.38276e+02,  2.30520e+02,  5.29759e+08],
       [-1.13413e+03,  2.09840e+02,  5.35350e+08],
       [-1.83527e+03,  1.45144e+02,  5.38410e+08],
       ...,
       [-1.56512e+03,  2.28173e+03,  5.88494e+08],
       [-1.17065e+03,  1.92441e+03,  5.81617e+08],
       [-1.62755e+03,  1.85961e+03,  5.86958e+08]])

In [26]:
env.observation_space.sample().shape

(210, 160, 3)

In [135]:
class Test:
    def __init__(self,a,b):
        self.a = a
        self.b = b
        
    def a_plus_b(self):
        return self.a+self.b
    def aaa(self):
        self.c = self.a + self.b

In [88]:
from pathlib import Path
initialDir = Path(CASE.initialDir())
ini = list(initialDir.glob("*"))

In [30]:
constant = Path(CASE.name + "/constant")
system = Path(CASE.name + "/system")
list(constant.glob("*"))

[PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/constant/transportProperties'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/constant/polyMesh'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/constant/turbulenceProperties'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/constant/g')]

In [34]:
a = list(system.glob("*"))

In [36]:
b = ParsedParameterFile(a[0])

In [85]:
os.path.basename(a[0])

'blockMeshDict'

In [ ]:
intialDir

In [89]:
ini

[PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/epsilon'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/k'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/alphat'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/polyMesh'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/T.orig'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/T'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/p'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/U'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/nut'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/p_rgh')]

In [5]:
# PyFoamで最適条件を探す用

class pyfoamtest:
    
    def __init__(self,CASE):
        
        self.CASE = CASE
        
        # get nCells
        with open (self.CASE.name + '/constant/polyMesh/neighbour') as f:
            neighbour = f.read()
        nCells_index = neighbour.find('nCells')
        nCells_ = neighbour[nCells_index : nCells_index+15]
        nCells = int(re.sub(r'\D', '', nCells_))
        self.nCells = nCells
        
        # 各辞書ファイルの取得
        self.initialDir = self.CASE.initialDir()+'/'
        self.constant = self.CASE.name + "/constant/"
        self.system = self.CASE.name + "/system/"
        self.initialDir_file = []
        for x in os.listdir(self.initialDir):
            if os.path.isfile(self.initialDir + x):
                self.initialDir_file.append(x)
        self.constant_file = []
        for y in os.listdir(self.constant):
            if os.path.isfile(self.constant + y):
                self.constant_file.append(y)
        self.system_file = []
        for z in os.listdir(self.system):
            if os.path.isfile(self.system + z):
                self.system_file.append(z)
        
        # 各辞書ファイルをそれぞれのファイル名で保存
        for i in range(len(self.initialDir_file)):
            self.__dict__[self.initialDir_file[i]] = ParsedParameterFile(self.initialDir + self.initialDir_file[i])
        for i in range(len(self.system_file)):
            self.__dict__[self.system_file[i]] = ParsedParameterFile(self.system + self.system_file[i])

In [42]:
test = pyfoamtest(CASE)

In [48]:
test.system_file

['blockMeshDict',
 'fvSolution',
 'blockMeshDict_save1',
 'createPatchDict',
 'refineMeshDict',
 'controlDict',
 'setFieldsDict',
 'fvSchemes']

In [46]:
test.fvSolution.getValueDict()

{'solvers': {'p_rgh': {'solver': 'PCG',
   'preconditioner': 'DIC',
   'tolerance': 1e-08,
   'relTol': 0.01},
  'p_rghFinal': {'$p_rgh': '', 'relTol': 0}},
 'PIMPLE': {'momentumPredictor': no,
  'nOuterCorrectors': 1,
  'nCorrectors': 2,
  'nNonOrthogonalCorrectors': 0,
  'pRefCell': 0,
  'pRefValue': 0},
 'relaxationFactors': {'equations': {}}}

In [71]:
test.epsilon.getValueDict()

{'dimensions': '[ 0 2 -3 0 0 0 0 ]',
 'internalField': 'uniform 0.01',
 'boundaryField': {'floor': {'type': 'epsilonWallFunction',
   'value': 'uniform 0.01'},
  'ceiling': {'type': 'epsilonWallFunction', 'value': 'uniform 0.01'},
  'fixedWalls': {'type': 'epsilonWallFunction', 'value': 'uniform 0.01'},
  'sideWalls': {'type': 'empty'},
  'outlet': {'type': 'zeroGradient'},
  'inlet': {'type': 'fixedValue', 'value': 'uniform 0.00082175'}}}

In [78]:
test.p_rgh.getValueDict()

{'dimensions': '[ 0 2 -3 0 0 0 0 ]',
 'internalField': 'uniform 0.01',
 'boundaryField': {'floor': {'type': 'epsilonWallFunction',
   'value': 'uniform 0.01'},
  'ceiling': {'type': 'epsilonWallFunction', 'value': 'uniform 0.01'},
  'fixedWalls': {'type': 'epsilonWallFunction', 'value': 'uniform 0.01'},
  'sideWalls': {'type': 'empty'},
  'outlet': {'type': 'zeroGradient'},
  'inlet': {'type': 'fixedValue', 'value': 'uniform 0.00082175'}}}

In [72]:
test.initialDir

[PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/epsilon'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/k'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/alphat'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/polyMesh'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/T.orig'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/T'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/p'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/U'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/nut'),
 PosixPath('/home/mmt3264/OpenFOAM/mmt3264-6/run/aircond2/0/p_rgh')]

In [33]:
# この環境の部分を変える
import copy
from box import Box

class Aircond:
    '''Aircondのクラス'''
    def __init__(self,CASE):
        self.CASE = CASE
        
        # get nCells
        with open (self.CASE.name + '/constant/polyMesh/neighbour') as f:
            neighbour = f.read()
        nCells_index = neighbour.find('nCells')
        nCells_ = neighbour[nCells_index : nCells_index+15]
        nCells = int(re.sub(r'\D', '', nCells_))
        self.nCells = nCells
        
        self.action_SPEED = np.array([0.1,0.3,0.5])
        self.action_DIRECTION = np.array([-1*np.pi/8, -2*np.pi/8,-3*np.pi/8])
        self.action_TEMPERTURE = np.array([18,22,26])
        self.action_space = np.tile(np.array([0,0,0]),(27,1))
        self.observation_space = np.tile(np.array([0,0,0]),(self.nCells,1))
        self.stride = 1200
        self.startTime = 0  # startTimeの初期化
        self.endTime = 1200  # 初期化
        self.end = 20000
        
        
        # 各辞書ファイルの取得
        self.initialDir = self.CASE.initialDir()+'/'
        self.constant = self.CASE.name + "/constant/"
        self.system = self.CASE.name + "/system/"
        self.initialDir_file = []
        for x in os.listdir(self.initialDir):
            if os.path.isfile(self.initialDir + x):
                self.initialDir_file.append(x)
        self.constant_file = []
        for y in os.listdir(self.constant):
            if os.path.isfile(self.constant + y):
                self.constant_file.append(y)
        self.system_file = []
        for z in os.listdir(self.system):
            if os.path.isfile(self.system + z):
                self.system_file.append(z)
        
        # 各辞書ファイルをそれぞれのファイル名で保存
        for i in range(len(self.initialDir_file)):
            self.__dict__[self.initialDir_file[i]] = ParsedParameterFile(self.initialDir + self.initialDir_file[i])

        for i in range(len(self.system_file)):
            self.__dict__[self.system_file[i]] = ParsedParameterFile(self.system + self.system_file[i])
            
    def initial_to_array(self, Parsed):
        '''uniformをnp.arrayに変換'''
        if Parsed.ndim==0:
            Parsed_raw = str(Parsed.all())
            Parsed_str = Parsed_raw[8:].strip('()').split(' ')
            Parsed_int = np.array(list(map(int,Parsed_str)))
            Parsed = np.tile(Parsed_int,(self.nCells,1))
        return Parsed

    def make_observation(self,Dir):
        '''Dirのpathのobservationを取得'''
        U_value = np.array(ParsedParameterFile(Dir + '/U').content['internalField'])
        T_value = np.array(ParsedParameterFile(Dir + '/T').content['internalField'])
        if U_value.ndim == 0:
            U_value = self.initial_to_array(U_value)
            T_value = self.initial_to_array(T_value)
        U_value_xy = np.delete(U_value, axis=1, obj=2)
        T_value_x = np.reshape(T_value, [-1,1], order='F')
        Observation = np.concatenate([U_value_xy, T_value_x],axis=1)
        return Observation    
    
    def make_action(self):
        '''actionの設定'''
        Action = np.empty((0,3),float)
        for i in range(len(self.action_SPEED)):
            for j in range(len(self.action_DIRECTION)):
                for k in range(len(self.action_TEMPERTURE)):
                    Ux = self.action_SPEED[i]*np.cos(self.action_DIRECTION[j])
                    Uy = self.action_SPEED[i]*np.sin(self.action_DIRECTION[j])
                    Act = np.array([[Ux,Uy,self.action_TEMPERTURE[k]]])
                    Action = np.append(Action,Act,axis=0)
                    
        return Action
        
    
    def reset(self):
        '''環境のリセット'''
        
        # reset control Dict
        clDict = ParsedParameterFile(self.CASE.controlDict())
        clDict['startTime'] = 0
        clDict['endTime'] = self.stride
        clDict['deltaT'] = 1
        clDict['writeInterval'] = 400
        clDict.writeFile()
        self.startTime = clDict['startTime']
        self.endTime = clDict['endTime']
        
        os.system('./Allclean')
        os.system('./Makemesh')
        
        # 初期条件の設定（ランダム）
        T_initial = ParsedParameterFile(self.CASE.initialDir() + '/T')
        #  random parameter from 26 to 35
        T_rand = np.random.randint(26+273,35+273)
        T_initial['internalField'].setUniform(T_rand)
        T_initial.writeFile()
        
        
        # set action and observation
        self.action_space= self.make_action()
        self.observation = self.make_observation(self.CASE.initialDir())
        return self.observation
    
    def step(self, action):
        '''ステップを進める'''
        
        clDict = ParsedParameterFile(self.CASE.controlDict())      
        if clDict['endTime'] == self.end:
            done = True
        else:
            done = False
            
            # actionに従った、境界条件を設定
            # action is 0~26
            U_latest = ParsedParameterFile(self.CASE.latestDir() + '/U')
            T_latest = ParsedParameterFile(self.CASE.latestDir() + '/T')
            self.act = self.action_space[action]
            U_latest['boundaryField']['inlet']['value'].setUniform(Vector(self.act[0],self.act[1],0))
            U_latest.writeFile()
            T_latest['boundaryField']['inlet']['value'].setUniform(self.act[2])
            T_latest.writeFile()
            
            # OpenFOAMのコマンドを実行
            args=shlex.split("buoyantBoussinesqPimpleFoam -case " + self.CASE.name)
            buoyant=BasicRunner(args,silent=True)
            self.summary=buoyant.start()
            runOK = buoyant.runOK()
            
            #os.system("buoyantBoussinesqPimpleFoam")
            
            # clDictのコントロール
            #clDict = ParsedParameterFile(self.CASE.controlDict())
            #clDict['startTime'] = self.startTime + self.stride
            #clDict['endTime'] = self.endTime + self.stride
            #clDict['deltaT'] = 1
            #clDict['writeInterval'] = 400
            #clDict.writeFile()
            
            self.startTime = clDict['startTime']
            self.endTime = clDict['endTime']
            
            self.observation = self.make_observation(self.CASE.latestDir())
            
        return (self.observation, done, runOK)
        

In [34]:
a = Aircond(CASE)

In [35]:
a.reset()

array([[  0,   0, 303],
       [  0,   0, 303],
       [  0,   0, 303],
       ...,
       [  0,   0, 303],
       [  0,   0, 303],
       [  0,   0, 303]])

In [32]:
a.step(12)

(array([[-5.22058e+11, -7.43962e+11,  2.99331e+02],
        [ 4.78941e+11,  7.77786e+11,  2.99332e+02],
        [-5.56393e+11, -7.61285e+11,  2.99334e+02],
        ...,
        [-3.79360e+11, -6.75051e+11,  2.99341e+02],
        [ 4.45513e+11,  6.60686e+11,  2.99341e+02],
        [-4.03820e+11, -6.77333e+11,  2.99341e+02]]), False, True)

In [210]:
a.startTime

8400

In [48]:
act = a.action[26]

In [54]:
act[0:2]

array([ 0.19134172, -0.46193977])

In [55]:
act[2]

26.0

In [62]:
v = Vector(0.2,0,0)

In [194]:
# control Dict
clDict = ParsedParameterFile(CASE.controlDict())
clDict['startTime'] = 0
clDict['endTime'] = 1200
clDict['deltaT'] = 2
clDict['writeInterval'] = 200
clDict.writeFile()

In [185]:
# OpenFOAMのコマンドを実行
args=shlex.split("buoyantBoussinesqPimpleFoam -case " + CASE.name)
buoyant=BasicRunner(args,silent=True)
summary=buoyant.start()
buoyant.runOK()

False

In [174]:
clDict['startTime']

1200

In [172]:
clDict['startTime'] = 1200
clDict['endTime'] = 2400
clDict['deltaT'] = 2
clDict['writeInterval'] = 200
clDict.writeFile()

In [38]:
# gym analysis
env = gym.make(ENV)
env.observation_space.sample()

array([4.6860966e-01, 1.4645028e+38, 8.6090848e-02, 3.0545910e+37],
      dtype=float32)

In [39]:
env.reset()

array([-0.03366172,  0.03967458, -0.01574906,  0.01677343])

In [22]:
observation = env.reset()
observation

array([-0.00977381, -0.04072757, -0.01555263, -0.0265637 ])

In [178]:
#action = agent.get_action(observation, episode)
observation_next,reward,done,info = env.step(1)

In [184]:
type(env.step(1))

tuple

In [180]:
reward

1.0

In [181]:
done

False

In [182]:
info

{}

In [10]:
# CartPoleを実行する環境のクラスです


class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める

                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                display_frames_as_gif(frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする


In [30]:
# CartPoleを実行する環境のクラスです


class Environment:

    def __init__(self):
        self.env = gym.make(ENV)  # 実行する課題を設定
        num_states = self.env.observation_space.shape[0]  # 課題の状態数4を取得
        num_actions = self.env.action_space.n  # CartPoleの行動（右に左に押す）の2を取得
        self.agent = Agent(num_states, num_actions)  # 環境内で行動するAgentを生成

        
    def run(self):
        '''実行'''
        episode_10_list = np.zeros(10)  # 10試行分の立ち続けたstep数を格納し、平均ステップ数を出力に利用
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        episode_final = False  # 最後の試行フラグ
        frames = []  # 最後の試行を動画にするために画像を格納する変数

        for episode in range(NUM_EPISODES):  # 最大試行数分繰り返す
            observation = self.env.reset()  # 環境の初期化

            state = observation  # 観測をそのまま状態sとして使用
            state = torch.from_numpy(state).type(
                torch.FloatTensor)  # NumPy変数をPyTorchのテンソルに変換
            state = torch.unsqueeze(state, 0)  # size 4をsize 1x4に変換

            for step in range(MAX_STEPS):  # 1エピソードのループ

                if episode_final is True:  # 最終試行ではframesに各時刻の画像を追加していく
                    frames.append(self.env.render(mode='rgb_array'))

                action = self.agent.get_action(state, episode)  # 行動を求める
                A ='''
                # 行動a_tの実行により、s_{t+1}とdoneフラグを求める
                # actionから.item()を指定して、中身を取り出す
                observation_next, _, done, _ = self.env.step(
                    action.item())  # rewardとinfoは使わないので_にする

                # 報酬を与える。さらにepisodeの終了評価と、state_nextを設定する
                if done:  # ステップ数が200経過するか、一定角度以上傾くとdoneはtrueになる
                    state_next = None  # 次の状態はないので、Noneを格納

                    # 直近10episodeの立てたstep数リストに追加
                    episode_10_list = np.hstack(
                        (episode_10_list[1:], step + 1))

                    if step < 195:
                        reward = torch.FloatTensor(
                            [-1.0])  # 途中でこけたら罰則として報酬-1を与える
                        complete_episodes = 0  # 連続成功記録をリセット
                    else:
                        reward = torch.FloatTensor([1.0])  # 立ったまま終了時は報酬1を与える
                        complete_episodes = complete_episodes + 1  # 連続記録を更新
                else:
                    reward = torch.FloatTensor([0.0])  # 普段は報酬0
                    state_next = observation_next  # 観測をそのまま状態とする
                    state_next = torch.from_numpy(state_next).type(
                        torch.FloatTensor)  # numpy変数をPyTorchのテンソルに変換
                    state_next = torch.unsqueeze(state_next, 0)  # size 4をsize 1x4に変換

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Experience ReplayでQ関数を更新する
                self.agent.update_q_function()

                # 観測の更新
                state = state_next

                # 終了時の処理
                if done:
                    print('%d Episode: Finished after %d steps：10試行の平均step数 = %.1lf' % (
                        episode, step + 1, episode_10_list.mean()))
                    break

            if episode_final is True:
                # 動画を保存と描画
                display_frames_as_gif(frames)
                break

            # 10連続で200step経ち続けたら成功
            if complete_episodes >= 10:
                print('10回連続成功')
                episode_final = True  # 次の試行を描画を行う最終試行とする
            '''
            
            return action

In [56]:
# main クラス
cartpole_env = Environment()
cartpole_env.run()


Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)


tensor([[1]])